# Robustness of Newton's Method

### _[Pile item](https://github.com/ArdanaLabs/audit/issues/4)_

The invariant equation from pg5 of the stableswap paper can be interpreted as a _polynomial function_ $f$ which is set to zero. 

At runtime of the solver, $n$ and all $x_i$ are known. The optimal value for parameter $A$ is discovered by simulation and known by the programmer. 

$$f = D \mapsto \frac{-1}{n^n \Pi x_i} D^{n+1} + (1 - A n^n) D + A n^n \Sigma x_i \\
f' = D \mapsto \frac{- (n + 1)}{n^n \Pi x_i} D^n + (1 - A n^n) \\
f'' = D \mapsto \frac{- (n + 1)}{n^{n-1} \Pi x_i} D^{n - 1}$$

Conveniently for us, $f' D = 0$ can be solved directly (as can $f'' D = 0$)
$$D = \left(\frac{(n^{-n} - A) n^{2n} \Pi x_i}{n + 1} \right)^{\frac{1}{n}}$$
where almost surely $A > n^{-n}$. As such, we have to think through all the implications of `nth` roots of negative numbers, recalling that $\forall x < 0, \exists x^{1/n} \in \mathbb{R} \iff \text{odd}\ n$ also having that $x^{1/n}$ is nonunique, and $\forall x < 0, x^{1/n} \in \mathbb{C}$ whenever $\text{even}\ n$ and also nonunique. 